Use Python to fetch all tickers (for example, XAUUSDc and USDJPYc) listed in the MT5 platform (this was already connected to Exness Broker Cent account), and their quotes, such as spread and more. Print it using a dataframe and write it using continous number of columns.

assuming:

MetaTrader 5 is already installed and logged in
The MT5 terminal is connected to Exness Cent
Python has access to the MetaTrader5 package

The script:

Connects to MT5
Fetches all symbols (tickers) available in Market Watch
Extracts quotes and trading metadata (bid, ask, spread, digits, contract size, etc.)
Stores everything in a Pandas DataFrame
Prints the DataFrame with a continuous numeric index
Keeps each metric as its own column (clean, analyzable layout)

In [7]:
import MetaTrader5 as mt5
import pandas as pd

# -----------------------------
# Initialize MT5
# -----------------------------
if not mt5.initialize():
    raise RuntimeError(f"MT5 init failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None or "Exness" not in account.company:
    mt5.shutdown()
    raise RuntimeError("MT5 is not connected to Exness")

# -----------------------------
# Fetch symbols
# -----------------------------
symbols = mt5.symbols_get()
if not symbols:
    mt5.shutdown()
    raise RuntimeError("No symbols retrieved")

rows = []

for sym in symbols:
    name = sym.name

    # Exness Cent filter
    if not name.endswith("c"):
        continue

    # Ensure visible
    mt5.symbol_select(name, True)

    info = mt5.symbol_info(name)
    tick = mt5.symbol_info_tick(name)

    if info is None or tick is None:
        continue

    spread_price = (tick.ask - tick.bid) if tick.ask and tick.bid else None

    rows.append({
        "symbol": name,
        "bid": tick.bid,
        "ask": tick.ask,
        "last": tick.last,
        "spread_points": info.spread,
        "spread_price": spread_price,
        "digits": info.digits,
        "contract_size": info.trade_contract_size,
        "tick_value": info.trade_tick_value,
        "tick_size": info.trade_tick_size,
        "volume_min": info.volume_min,
        "volume_max": info.volume_max,
        "volume_step": info.volume_step,
        "swap_long": info.swap_long,
        "swap_short": info.swap_short,
        "trade_mode": info.trade_mode,
        "margin_initial": info.margin_initial,
        "margin_maintenance": info.margin_maintenance
    })

# -----------------------------
# Build DataFrame
# -----------------------------
df = pd.DataFrame(rows).reset_index(drop=True)

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 300)

print(df)

mt5.shutdown()


     symbol          bid          ask  last  spread_points  spread_price  digits  contract_size     tick_value  tick_size    volume_min    volume_max   volume_step  swap_long  swap_short  trade_mode  margin_initial  margin_maintenance
0   BTCUSDc  86562.99000  86580.99000   0.0           1800      18.00000       2           0.01       0.010000    0.01000  1.000000e-02  1.000000e+03  1.000000e-02    -1551.1         0.0           4             0.0                 0.0
1   AUDCADc      0.90977      0.90999   0.0             22       0.00022       5        1000.00       0.725695    0.00001  1.000000e-02  2.000000e+02  1.000000e-02       -1.4        -7.4           4             0.0                 0.0
2   AUDCHFc      0.52518      0.52527   0.0              9       0.00009       5        1000.00       1.257371    0.00001  1.000000e-02  2.000000e+02  1.000000e-02        0.0        -8.2           4             0.0                 0.0
3   AUDJPYc    102.84800    102.86700   0.0             19  

True

In [1]:
4333.832-4333.992

-0.15999999999985448

all Exness Cent symbols (*c) but exclude any non-tradable, indicator, or synthetic symbols, e.g., “forex_indicator” type instruments.

In MT5, such symbols usually have SYMBOL_TRADE_MODE_NONE or SYMBOL_TRADE_MODE_DISABLED and should be filtered out. Here’s a precise script:

In [28]:
import MetaTrader5 as mt5
import pandas as pd

# -----------------------------
# Initialize MT5
# -----------------------------
if not mt5.initialize():
    raise RuntimeError(f"MT5 init failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None or "Exness" not in account.company:
    mt5.shutdown()
    raise RuntimeError("MT5 is not connected to Exness")

# -----------------------------
# Fetch symbols
# -----------------------------
symbols = mt5.symbols_get()
if not symbols:
    mt5.shutdown()
    raise RuntimeError("No symbols retrieved")

rows = []

for sym in symbols:
    name = sym.name

    # Only Exness Cent symbols
    if not name.endswith("c"):
        continue

    # Skip non-tradable / indicator symbols
    info = mt5.symbol_info(name)
    if info is None or info.trade_mode != mt5.SYMBOL_TRADE_MODE_FULL:
        continue

    tick = mt5.symbol_info_tick(name)
    if tick is None:
        continue

    spread_price = (tick.ask - tick.bid) if tick.ask and tick.bid else None

    rows.append({
        "symbol": name,
        "bid": tick.bid,
        "ask": tick.ask,
        "last": tick.last,
        "spread_points": info.spread,
        "spread_price": spread_price,
        "digits": info.digits,
        "contract_size": info.trade_contract_size,
        "tick_value": info.trade_tick_value,
        "tick_size": info.trade_tick_size,
        "volume_min": info.volume_min,
        "volume_max": info.volume_max,
        "volume_step": info.volume_step,
        "swap_long": info.swap_long,
        "swap_short": info.swap_short,
        "margin_initial": info.margin_initial,
        "margin_maintenance": info.margin_maintenance
    })

# -----------------------------
# Build DataFrame
# -----------------------------
df = pd.DataFrame(rows).reset_index(drop=True)

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 300)

print(df)

mt5.shutdown()


     symbol         bid         ask    last  spread_points  spread_price  digits  contract_size  tick_value  tick_size  volume_min  volume_max  volume_step   swap_long  swap_short  margin_initial  margin_maintenance
0   BTCUSDc 86409.82000 86427.82000 0.00000           1800      18.00000       2        0.01000     0.01000    0.01000     0.01000  1000.00000      0.01000 -1551.10000     0.00000         0.00000             0.00000
1   AUDCADc     0.90973     0.90995 0.00000             22       0.00022       5     1000.00000     0.72589    0.00001     0.01000   200.00000      0.01000    -1.40000    -7.40000         0.00000             0.00000
2   AUDCHFc     0.52517     0.52526 0.00000              9       0.00009       5     1000.00000     1.25759    0.00001     0.01000   200.00000      0.01000     0.00000    -8.20000         0.00000             0.00000
3   AUDJPYc   102.95700   102.97600 0.00000             19       0.01900       3     1000.00000     0.64149    0.00100     0.01000   200

True

In [ ]:
tick distance needed to blow -1 USC

In [45]:
import MetaTrader5 as mt5
import pandas as pd

# -----------------------------
# Initialize MT5
# -----------------------------
if not mt5.initialize():
    raise RuntimeError(f"MT5 init failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None or "Exness" not in account.company:
    mt5.shutdown()
    raise RuntimeError("MT5 is not connected to Exness")

# -----------------------------
# Fetch symbols
# -----------------------------
symbols = mt5.symbols_get()
if not symbols:
    mt5.shutdown()
    raise RuntimeError("No symbols retrieved")

rows = []

for sym in symbols:
    name = sym.name

    # Only Exness Cent symbols
    if not name.endswith("c"):
        continue

    # Skip non-tradable / indicator symbols
    info = mt5.symbol_info(name)
    if info is None or info.trade_mode != mt5.SYMBOL_TRADE_MODE_FULL:
        continue

    tick = mt5.symbol_info_tick(name)
    if tick is None:
        continue

    spread_price = (tick.ask - tick.bid) if tick.ask and tick.bid else None

    # Round prices according to the symbol digits
    digits = info.digits
    bid = round(tick.bid, digits) if tick.bid is not None else None
    ask = round(tick.ask, digits) if tick.ask is not None else None
    last = round(tick.last, digits) if tick.last is not None else None
    spread_price = round(spread_price, digits) if spread_price is not None else None

    rows.append({
        "symbol": name,
        "bid": bid,
        "ask": ask,
        "last": last,
        "spread_points": info.spread,
        "spread_price": spread_price,
        "digits": digits,
        "contract_size": info.trade_contract_size,
        "tick_value": info.trade_tick_value,
        "tick_size": info.trade_tick_size,
        "volume_min": info.volume_min,
        "volume_max": info.volume_max,
        "volume_step": info.volume_step,
        "swap_long": info.swap_long,
        "swap_short": info.swap_short,
        "margin_initial": info.margin_initial,
        "margin_maintenance": info.margin_maintenance
    })

# -----------------------------
# Build DataFrame
# -----------------------------
df = pd.DataFrame(rows).reset_index(drop=True)

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 300)

print(df)

mt5.shutdown()


     symbol         bid         ask    last  spread_points  spread_price  digits  contract_size  tick_value  tick_size  volume_min  volume_max  volume_step   swap_long  swap_short  margin_initial  margin_maintenance
0   BTCUSDc 86773.18000 86791.18000 0.00000           1800      18.00000       2        0.01000     0.01000    0.01000     0.01000  1000.00000      0.01000 -1551.10000     0.00000         0.00000             0.00000
1   AUDCADc     0.90975     0.90997 0.00000             22       0.00022       5     1000.00000     0.72622    0.00001     0.01000   200.00000      0.01000    -1.40000    -7.40000         0.00000             0.00000
2   AUDCHFc     0.52545     0.52554 0.00000              9       0.00009       5     1000.00000     1.25753    0.00001     0.01000   200.00000      0.01000     0.00000    -8.20000         0.00000             0.00000
3   AUDJPYc   103.02700   103.04600 0.00000             19       0.01900       3     1000.00000     0.64140    0.00100     0.01000   200

True

In [51]:
import MetaTrader5 as mt5
import pandas as pd

# -----------------------------
# Initialize MT5
# -----------------------------
if not mt5.initialize():
    raise RuntimeError(f"MT5 init failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None or "Exness" not in account.company:
    mt5.shutdown()
    raise RuntimeError("MT5 is not connected to Exness")

# -----------------------------
# Fetch symbols
# -----------------------------
symbols = mt5.symbols_get()
if not symbols:
    mt5.shutdown()
    raise RuntimeError("No symbols retrieved")

rows = []

for sym in symbols:
    name = sym.name

    # Only Exness Cent symbols
    if not name.endswith("c"):
        continue

    # Skip non-tradable / indicator symbols
    info = mt5.symbol_info(name)
    if info is None or info.trade_mode != mt5.SYMBOL_TRADE_MODE_FULL:
        continue

    tick = mt5.symbol_info_tick(name)
    if tick is None:
        continue

    digits = info.digits

    # Format all relevant numeric fields as strings with correct digits
    bid = f"{tick.bid:.{digits}f}" if tick.bid is not None else None
    ask = f"{tick.ask:.{digits}f}" if tick.ask is not None else None
    spread_price = f"{(tick.ask - tick.bid):.{digits}f}" if tick.ask is not None and tick.bid is not None else None

    contract_size = f"{info.trade_contract_size:.{digits}f}" if info.trade_contract_size is not None else None
    tick_value = f"{info.trade_tick_value:.{digits}f}" if info.trade_tick_value is not None else None
    tick_size = f"{info.trade_tick_size:.{digits}f}" if info.trade_tick_size is not None else None
    volume_min = f"{info.volume_min:.{digits}f}" if info.volume_min is not None else None
    volume_max = f"{info.volume_max:.{digits}f}" if info.volume_max is not None else None
    volume_step = f"{info.volume_step:.{digits}f}" if info.volume_step is not None else None
    swap_long = f"{info.swap_long:.{digits}f}" if info.swap_long is not None else None
    swap_short = f"{info.swap_short:.{digits}f}" if info.swap_short is not None else None

    rows.append({
        "symbol": name,
        "bid": bid,
        "ask": ask,
        "spread_points": info.spread,
        "spread_price": spread_price,
        "digits": digits,
        "contract_size": contract_size,
        "tick_value": tick_value,
        "tick_size": tick_size,
        "volume_min": volume_min,
        "volume_max": volume_max,
        "volume_step": volume_step,
        "swap_long": swap_long,
        "swap_short": swap_short
    })

# -----------------------------
# Build DataFrame
# -----------------------------
df = pd.DataFrame(rows).reset_index(drop=True)

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 300)

print(df)

mt5.shutdown()


     symbol       bid       ask  spread_points spread_price  digits contract_size tick_value tick_size volume_min volume_max volume_step  swap_long swap_short
0   BTCUSDc  86791.67  86809.67           1800        18.00       2          0.01       0.01      0.01       0.01    1000.00        0.01   -1551.10       0.00
1   AUDCADc   0.90978   0.91000             22      0.00022       5    1000.00000    0.72608   0.00001    0.01000  200.00000     0.01000   -1.40000   -7.40000
2   AUDCHFc   0.52537   0.52546              9      0.00009       5    1000.00000    1.25750   0.00001    0.01000  200.00000     0.01000    0.00000   -8.20000
3   AUDJPYc   103.027   103.046             19        0.019       3      1000.000      0.641     0.001      0.010    200.000       0.010     -0.200     -1.900
4   AUDNZDc   1.14642   1.14662             20      0.00020       5    1000.00000    0.57625   0.00001    0.01000  200.00000     0.01000   -1.40000   -8.50000
5   AUDUSDc   0.66073   0.66082              9

True

In [52]:
import MetaTrader5 as mt5
import pandas as pd

# -----------------------------
# Initialize MT5
# -----------------------------
if not mt5.initialize():
    raise RuntimeError(f"MT5 init failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None or "Exness" not in account.company:
    mt5.shutdown()
    raise RuntimeError("MT5 is not connected to Exness")

# -----------------------------
# Fetch symbols
# -----------------------------
symbols = mt5.symbols_get()
if not symbols:
    mt5.shutdown()
    raise RuntimeError("No symbols retrieved")

rows = []

risk_per_trade = 1.0       # 1 USC
trade_volume = 0.01         # lots

for sym in symbols:
    name = sym.name

    # Only Exness Cent symbols
    if not name.endswith("c"):
        continue

    # Skip non-tradable / indicator symbols
    info = mt5.symbol_info(name)
    if info is None or info.trade_mode != mt5.SYMBOL_TRADE_MODE_FULL:
        continue

    tick = mt5.symbol_info_tick(name)
    if tick is None:
        continue

    digits = info.digits

    # Format all relevant numeric fields
    bid = tick.bid
    ask = tick.ask
    spread_price = tick.ask - tick.bid if tick.ask is not None and tick.bid is not None else None

    contract_size = info.trade_contract_size
    tick_value = info.trade_tick_value
    tick_size = info.trade_tick_size

    # Open price reference
    open_price = bid

    # Stoploss distance for -1 USC risk
    if contract_size and tick_value:
        # Distance in price points
        distance = risk_per_trade / (trade_volume * contract_size * tick_value)
        # For buy trade, stoploss below open, for sell trade, above open
        stoploss_price = open_price - distance
    else:
        distance = None
        stoploss_price = None

    rows.append({
        "symbol": name,
        "bid": f"{bid:.{digits}f}" if bid is not None else None,
        "ask": f"{ask:.{digits}f}" if ask is not None else None,
        "spread_points": info.spread,
        "spread_price": f"{spread_price:.{digits}f}" if spread_price is not None else None,
        "digits": digits,
        "contract_size": f"{contract_size:.{digits}f}" if contract_size is not None else None,
        "tick_value": f"{tick_value:.{digits}f}" if tick_value is not None else None,
        "tick_size": f"{tick_size:.{digits}f}" if tick_size is not None else None,
        "volume_min": f"{info.volume_min:.{digits}f}" if info.volume_min is not None else None,
        "volume_max": f"{info.volume_max:.{digits}f}" if info.volume_max is not None else None,
        "volume_step": f"{info.volume_step:.{digits}f}" if info.volume_step is not None else None,
        "swap_long": f"{info.swap_long:.{digits}f}" if info.swap_long is not None else None,
        "swap_short": f"{info.swap_short:.{digits}f}" if info.swap_short is not None else None,
        "open_price": f"{open_price:.{digits}f}" if open_price is not None else None,
        "stoploss_price": f"{stoploss_price:.{digits}f}" if stoploss_price is not None else None,
        "distance": f"{distance:.{digits}f}" if distance is not None else None
    })

# -----------------------------
# Build DataFrame
# -----------------------------
df = pd.DataFrame(rows).reset_index(drop=True)

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 300)

print(df)

mt5.shutdown()


     symbol       bid       ask  spread_points spread_price  digits contract_size tick_value tick_size volume_min volume_max volume_step  swap_long swap_short open_price stoploss_price    distance
0   BTCUSDc  86686.56  86704.56           1800        18.00       2          0.01       0.01      0.01       0.01    1000.00        0.01   -1551.10       0.00   86686.56     -913313.44  1000000.00
1   AUDCADc   0.91009   0.91031             22      0.00022       5    1000.00000    0.72610   0.00001    0.01000  200.00000     0.01000   -1.40000   -7.40000    0.91009        0.77237     0.13772
2   AUDCHFc   0.52564   0.52573              9      0.00009       5    1000.00000    1.25734   0.00001    0.01000  200.00000     0.01000    0.00000   -8.20000    0.52564        0.44611     0.07953
3   AUDJPYc   103.068   103.087             19        0.019       3      1000.000      0.641     0.001      0.010    200.000       0.010     -0.200     -1.900    103.068        102.912       0.156
4   AUDNZDc   1

True

In [60]:
import pandas as pd

# Title
title = "How much 'distance needed' equivalent of 1 unit risk (account's money as unit. It's 1 USC (cent) or 1 USD (dollar) in Exness Broker) if using a 0.01 volume lot size."

# Data
data = {
    "symbol": [
        "BTCUSDc","AUDCADc","AUDCHFc","AUDJPYc","AUDNZDc","AUDUSDc","CADJPYc","CHFJPYc",
        "EURAUDc","EURCADc","EURCHFc","EURGBPc","EURJPYc","EURNZDc","EURUSDc","GBPAUDc",
        "GBPCADc","GBPCHFc","GBPJPYc","GBPNZDc","GBPUSDc","NZDJPYc","NZDUSDc","USDCADc",
        "USDCHFc","USDHKDc","USDJPYc","XAGUSDc","XAUUSDc"
    ],
    "needed_tick_points": [
        10000,138,80,156,173,100,156,156,151,138,80,75,156,174,100,151,138,80,156,174,100,156,100,138,80,778,156,20,1000
    ],
    "needed_pips_points": [
        1000,13.8,8,15.6,17.3,10,15.6,15.6,15.1,13.8,8,7.5,15.6,17.4,10,15.1,13.8,8,15.6,17.4,10,15.6,10,13.8,8,77.8,15.6,2,100
    ],
    "needed_symbol_price_points": [
        100,0.00138,0.0008,0.156,0.00173,0.001,0.156,0.156,0.00151,0.00138,0.0008,0.00075,0.156,0.00174,0.001,0.00151,
        0.00138,0.0008,0.156,0.00174,0.001,0.156,0.001,0.00138,0.0008,0.00778,0.156,0.02,1
    ]
}

# Create DataFrame
df = pd.DataFrame(data)

# Print title and DataFrame
print(title)
print(df)


How much 'distance needed' equivalent of 1 unit risk (account's money as unit. It's 1 USC (cent) or 1 USD (dollar) in Exness Broker) if using a 0.01 volume lot size.
     symbol  needed_tick_points  needed_pips_points  needed_symbol_price_points
0   BTCUSDc               10000          1000.00000                   100.00000
1   AUDCADc                 138            13.80000                     0.00138
2   AUDCHFc                  80             8.00000                     0.00080
3   AUDJPYc                 156            15.60000                     0.15600
4   AUDNZDc                 173            17.30000                     0.00173
5   AUDUSDc                 100            10.00000                     0.00100
6   CADJPYc                 156            15.60000                     0.15600
7   CHFJPYc                 156            15.60000                     0.15600
8   EURAUDc                 151            15.10000                     0.00151
9   EURCADc                 138   

In [62]:
import MetaTrader5 as mt5
import pandas as pd

# -----------------------------
# Title
# -----------------------------
title = "How much 'distance needed' equivalent of 1 unit risk (account's money as unit. It's 1 USC (cent) or 1 USD (dollar) in Exness Broker) if using a 0.01 volume lot size."

# -----------------------------
# Initialize MT5
# -----------------------------
if not mt5.initialize():
    raise RuntimeError(f"MT5 init failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None or "Exness" not in account.company:
    mt5.shutdown()
    raise RuntimeError("MT5 is not connected to Exness")

# -----------------------------
# Pre-calculated needed distances (matches your static example)
# -----------------------------
symbols_needed = {
    "symbol": [
        "BTCUSDc","AUDCADc","AUDCHFc","AUDJPYc","AUDNZDc","AUDUSDc","CADJPYc","CHFJPYc",
        "EURAUDc","EURCADc","EURCHFc","EURGBPc","EURJPYc","EURNZDc","EURUSDc","GBPAUDc",
        "GBPCADc","GBPCHFc","GBPJPYc","GBPNZDc","GBPUSDc","NZDJPYc","NZDUSDc","USDCADc",
        "USDCHFc","USDHKDc","USDJPYc","XAGUSDc","XAUUSDc"
    ],
    "needed_tick_points": [
        10000,138,80,156,173,100,156,156,151,138,80,75,156,174,100,151,138,80,156,174,100,156,100,138,80,778,156,20,1000
    ],
    "needed_pips_points": [
        1000,13.8,8,15.6,17.3,10,15.6,15.6,15.1,13.8,8,7.5,15.6,17.4,10,15.1,13.8,8,15.6,17.4,10,15.6,10,13.8,8,77.8,15.6,2,100
    ],
    "needed_symbol_price_points": [
        100,0.00138,0.0008,0.156,0.00173,0.001,0.156,0.156,0.00151,0.00138,0.0008,0.00075,0.156,0.00174,0.001,0.00151,
        0.00138,0.0008,0.156,0.00174,0.001,0.156,0.001,0.00138,0.0008,0.00778,0.156,0.02,1
    ]
}

# -----------------------------
# Build DataFrame
# -----------------------------
df_needed = pd.DataFrame(symbols_needed)

# -----------------------------
# Add live MT5 bid/ask data for these symbols
# -----------------------------
live_data = []

for sym in df_needed["symbol"]:
    info = mt5.symbol_info(sym)
    tick = mt5.symbol_info_tick(sym)

    if info is None or tick is None:
        live_data.append({
            "bid": None,
            "ask": None,
            "spread_points": None
        })
        continue

    bid = tick.bid
    ask = tick.ask
    spread = info.spread if info.spread is not None else (ask - bid if ask and bid else None)

    live_data.append({
        "bid": bid,
        "ask": ask,
        "spread_points": spread
    })

df_live = pd.DataFrame(live_data)

# -----------------------------
# Combine static needed distances with live MT5 data
# -----------------------------
df = pd.concat([df_needed, df_live], axis=1)

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 300)

# -----------------------------
# Print everything
# -----------------------------
print(title)
print(df)

# -----------------------------
# Shutdown MT5
# -----------------------------
mt5.shutdown()


How much 'distance needed' equivalent of 1 unit risk (account's money as unit. It's 1 USC (cent) or 1 USD (dollar) in Exness Broker) if using a 0.01 volume lot size.
     symbol  needed_tick_points  needed_pips_points  needed_symbol_price_points         bid         ask  spread_points
0   BTCUSDc               10000          1000.00000                   100.00000 86822.98000 86840.98000           1800
1   AUDCADc                 138            13.80000                     0.00138     0.91032     0.91054             22
2   AUDCHFc                  80             8.00000                     0.00080     0.52590     0.52599              9
3   AUDJPYc                 156            15.60000                     0.15600   102.99100   103.01000             19
4   AUDNZDc                 173            17.30000                     0.00173     1.14649     1.14669             20
5   AUDUSDc                 100            10.00000                     0.00100     0.66087     0.66096              9
6

True

In [63]:
import MetaTrader5 as mt5
import pandas as pd

# -----------------------------
# Title
# -----------------------------
title = "How much 'distance needed' equivalent of 1 unit risk (account's money as unit. It's 1 USC (cent) or 1 USD (dollar) in Exness Broker) if using a 0.01 volume lot size."

# -----------------------------
# Initialize MT5
# -----------------------------
if not mt5.initialize():
    raise RuntimeError(f"MT5 init failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None or "Exness" not in account.company:
    mt5.shutdown()
    raise RuntimeError("MT5 is not connected to Exness")

# -----------------------------
# Pre-calculated needed distances
# -----------------------------
symbols_needed = {
    "symbol": [
        "BTCUSDc","AUDCADc","AUDCHFc","AUDJPYc","AUDNZDc","AUDUSDc","CADJPYc","CHFJPYc",
        "EURAUDc","EURCADc","EURCHFc","EURGBPc","EURJPYc","EURNZDc","EURUSDc","GBPAUDc",
        "GBPCADc","GBPCHFc","GBPJPYc","GBPNZDc","GBPUSDc","NZDJPYc","NZDUSDc","USDCADc",
        "USDCHFc","USDHKDc","USDJPYc","XAGUSDc","XAUUSDc"
    ],
    "needed_tick_points": [
        10000,138,80,156,173,100,156,156,151,138,80,75,156,174,100,151,138,80,156,174,100,156,100,138,80,778,156,20,1000
    ],
    "needed_pips_points": [
        1000,13.8,8,15.6,17.3,10,15.6,15.6,15.1,13.8,8,7.5,15.6,17.4,10,15.1,13.8,8,15.6,17.4,10,15.6,10,13.8,8,77.8,15.6,2,100
    ],
    "needed_symbol_price_points": [
        100,0.00138,0.0008,0.156,0.00173,0.001,0.156,0.156,0.00151,0.00138,0.0008,0.00075,0.156,0.00174,0.001,0.00151,
        0.00138,0.0008,0.156,0.00174,0.001,0.156,0.001,0.00138,0.0008,0.00778,0.156,0.02,1
    ]
}

df_needed = pd.DataFrame(symbols_needed)

# -----------------------------
# Add live MT5 data including requested columns
# -----------------------------
live_data = []

for sym in df_needed["symbol"]:
    info = mt5.symbol_info(sym)
    tick = mt5.symbol_info_tick(sym)

    if info is None or tick is None:
        live_data.append({
            "bid": None,
            "ask": None,
            "spread_points": None,
            "spread_price": None,
            "digits": None,
            "volume_min": None,
            "volume_max": None,
            "volume_step": None,
            "swap_long": None,
            "swap_short": None
        })
        continue

    bid = tick.bid
    ask = tick.ask
    digits = info.digits
    spread_points = info.spread if info.spread is not None else (ask - bid if ask and bid else None)
    spread_price = (ask - bid) if ask and bid else None

    live_data.append({
        "bid": bid,
        "ask": ask,
        "spread_points": spread_points,
        "spread_price": spread_price,
        "digits": digits,
        "volume_min": info.volume_min,
        "volume_max": info.volume_max,
        "volume_step": info.volume_step,
        "swap_long": info.swap_long,
        "swap_short": info.swap_short
    })

df_live = pd.DataFrame(live_data)

# -----------------------------
# Combine pre-calculated distances with live data
# -----------------------------
df = pd.concat([df_needed, df_live], axis=1)

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 300)

# -----------------------------
# Print everything
# -----------------------------
print(title)
print(df)

# -----------------------------
# Shutdown MT5
# -----------------------------
mt5.shutdown()


How much 'distance needed' equivalent of 1 unit risk (account's money as unit. It's 1 USC (cent) or 1 USD (dollar) in Exness Broker) if using a 0.01 volume lot size.
     symbol  needed_tick_points  needed_pips_points  needed_symbol_price_points         bid         ask  spread_points  spread_price  digits  volume_min  volume_max  volume_step   swap_long  swap_short
0   BTCUSDc               10000          1000.00000                   100.00000 86847.95000 86865.95000           1800      18.00000       2     0.01000  1000.00000      0.01000 -1551.10000     0.00000
1   AUDCADc                 138            13.80000                     0.00138     0.91019     0.91041             22       0.00022       5     0.01000   200.00000      0.01000    -1.40000    -7.40000
2   AUDCHFc                  80             8.00000                     0.00080     0.52586     0.52595              9       0.00009       5     0.01000   200.00000      0.01000     0.00000    -8.20000
3   AUDJPYc               

True

In [64]:
import MetaTrader5 as mt5
import pandas as pd

# -----------------------------
# Title
# -----------------------------
title = "How much 'distance needed' equivalent of 1 unit risk (account's money as unit. It's 1 USC (cent) or 1 USD (dollar) in Exness Broker) if using a 0.01 volume lot size."

# -----------------------------
# Initialize MT5
# -----------------------------
if not mt5.initialize():
    raise RuntimeError(f"MT5 init failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None or "Exness" not in account.company:
    mt5.shutdown()
    raise RuntimeError("MT5 is not connected to Exness")

# -----------------------------
# Pre-calculated needed distances
# -----------------------------
symbols_needed = {
    "symbol": [
        "BTCUSDc","AUDCADc","AUDCHFc","AUDJPYc","AUDNZDc","AUDUSDc","CADJPYc","CHFJPYc",
        "EURAUDc","EURCADc","EURCHFc","EURGBPc","EURJPYc","EURNZDc","EURUSDc","GBPAUDc",
        "GBPCADc","GBPCHFc","GBPJPYc","GBPNZDc","GBPUSDc","NZDJPYc","NZDUSDc","USDCADc",
        "USDCHFc","USDHKDc","USDJPYc","XAGUSDc","XAUUSDc"
    ],
    "needed_tick_points": [
        10000,138,80,156,173,100,156,156,151,138,80,75,156,174,100,151,138,80,156,174,100,156,100,138,80,778,156,20,1000
    ],
    "needed_pips_points": [
        1000,13.8,8,15.6,17.3,10,15.6,15.6,15.1,13.8,8,7.5,15.6,17.4,10,15.1,13.8,8,15.6,17.4,10,15.6,10,13.8,8,77.8,15.6,2,100
    ],
    "needed_symbol_price_points": [
        100,0.00138,0.0008,0.156,0.00173,0.001,0.156,0.156,0.00151,0.00138,0.0008,0.00075,0.156,0.00174,0.001,0.00151,
        0.00138,0.0008,0.156,0.00174,0.001,0.156,0.001,0.00138,0.0008,0.00778,0.156,0.02,1
    ]
}

df_needed = pd.DataFrame(symbols_needed)

# -----------------------------
# Add live MT5 data including requested columns
# -----------------------------
live_data = []

for sym in df_needed["symbol"]:
    info = mt5.symbol_info(sym)
    tick = mt5.symbol_info_tick(sym)

    if info is None or tick is None:
        live_data.append({
            "bid": None,
            "ask": None,
            "spread_tick_points": None,
            "spread_price_points": None,
            "digits": None,
            "volume_min": None,
            "volume_max": None,
            "volume_step": None,
            "swap_long": None,
            "swap_short": None
        })
        continue

    bid = tick.bid
    ask = tick.ask
    digits = info.digits
    spread_tick = info.spread if info.spread is not None else (ask - bid if ask and bid else None)
    spread_price = (ask - bid) if ask and bid else None

    live_data.append({
        "bid": bid,
        "ask": ask,
        "spread_tick_points": spread_tick,
        "spread_price_points": spread_price,
        "digits": digits,
        "volume_min": info.volume_min,
        "volume_max": info.volume_max,
        "volume_step": info.volume_step,
        "swap_long": info.swap_long,
        "swap_short": info.swap_short
    })

df_live = pd.DataFrame(live_data)

# -----------------------------
# Combine pre-calculated distances with live data
# -----------------------------
df = pd.concat([df_needed, df_live], axis=1)

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 300)

# -----------------------------
# Print everything
# -----------------------------
print(title)
print(df)

# -----------------------------
# Shutdown MT5
# -----------------------------
mt5.shutdown()


How much 'distance needed' equivalent of 1 unit risk (account's money as unit. It's 1 USC (cent) or 1 USD (dollar) in Exness Broker) if using a 0.01 volume lot size.
     symbol  needed_tick_points  needed_pips_points  needed_symbol_price_points         bid         ask  spread_tick_points  spread_price_points  digits  volume_min  volume_max  volume_step   swap_long  swap_short
0   BTCUSDc               10000          1000.00000                   100.00000 86781.02000 86799.02000                1800             18.00000       2     0.01000  1000.00000      0.01000 -1551.10000     0.00000
1   AUDCADc                 138            13.80000                     0.00138     0.90996     0.91018                  22              0.00022       5     0.01000   200.00000      0.01000    -1.40000    -7.40000
2   AUDCHFc                  80             8.00000                     0.00080     0.52576     0.52585                   9              0.00009       5     0.01000   200.00000      0.01000   

True

In [66]:
import MetaTrader5 as mt5
import pandas as pd

# -----------------------------
# Title
# -----------------------------
title = "How much 'distance needed' equivalent of 1 unit risk (account's money as unit. It's 1 USC (cent, for cent accounts) or 1 USD (dollar, for dollar accounts) in Exness Broker) if using a 0.01 volume lot size."

# -----------------------------
# Initialize MT5
# -----------------------------
if not mt5.initialize():
    raise RuntimeError(f"MT5 init failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None or "Exness" not in account.company:
    mt5.shutdown()
    raise RuntimeError("MT5 is not connected to Exness")

# -----------------------------
# Pre-calculated needed distances
# -----------------------------
symbols_needed = {
    "symbol": [
        "BTCUSDc","AUDCADc","AUDCHFc","AUDJPYc","AUDNZDc","AUDUSDc","CADJPYc","CHFJPYc",
        "EURAUDc","EURCADc","EURCHFc","EURGBPc","EURJPYc","EURNZDc","EURUSDc","GBPAUDc",
        "GBPCADc","GBPCHFc","GBPJPYc","GBPNZDc","GBPUSDc","NZDJPYc","NZDUSDc","USDCADc",
        "USDCHFc","USDHKDc","USDJPYc","XAGUSDc","XAUUSDc"
    ],
    "needed_tick_points": [
        10000,138,80,156,173,100,156,156,151,138,80,75,156,174,100,151,138,80,156,174,100,156,100,138,80,778,156,20,1000
    ],
    "needed_pips_points": [
        1000,13.8,8,15.6,17.3,10,15.6,15.6,15.1,13.8,8,7.5,15.6,17.4,10,15.1,13.8,8,15.6,17.4,10,15.6,10,13.8,8,77.8,15.6,2,100
    ],
    "needed_symbol_price_points": [
        100,0.00138,0.0008,0.156,0.00173,0.001,0.156,0.156,0.00151,0.00138,0.0008,0.00075,0.156,0.00174,0.001,0.00151,
        0.00138,0.0008,0.156,0.00174,0.001,0.156,0.001,0.00138,0.0008,0.00778,0.156,0.02,1
    ]
}

df_needed = pd.DataFrame(symbols_needed)

# -----------------------------
# Add live MT5 data including requested columns
# -----------------------------
live_data = []

for i, sym in enumerate(df_needed["symbol"]):
    info = mt5.symbol_info(sym)
    tick = mt5.symbol_info_tick(sym)

    if info is None or tick is None:
        live_data.append({
            "bid": None,
            "ask": None,
            "spread_tick_points": None,
            "spread_price_points": None,
            "spread_pips_points": None,
            "digits": None,
            "volume_min": None,
            "volume_max": None,
            "volume_step": None,
            "swap_long": None,
            "swap_short": None
        })
        continue

    bid = tick.bid
    ask = tick.ask
    digits = info.digits
    spread_tick = info.spread if info.spread is not None else (ask - bid if ask and bid else None)
    spread_price = (ask - bid) if ask and bid else None

    # Calculate spread in pips points
    # 1 pip in points = needed_symbol_price_points[i] / needed_pips_points[i]
    # spread_pips_points = spread_price / 1 pip value
    pip_value = df_needed.loc[i, "needed_symbol_price_points"] / df_needed.loc[i, "needed_pips_points"]
    spread_pips_points = spread_price / pip_value if spread_price is not None else None

    live_data.append({
        "bid": bid,
        "ask": ask,
        "spread_tick_points": spread_tick,
        "spread_price_points": spread_price,
        "spread_pips_points": spread_pips_points,
        "digits": digits,
        "volume_min": info.volume_min,
        "volume_max": info.volume_max,
        "volume_step": info.volume_step,
        "swap_long": info.swap_long,
        "swap_short": info.swap_short
    })

df_live = pd.DataFrame(live_data)

# -----------------------------
# Combine pre-calculated distances with live data
# -----------------------------
df = pd.concat([df_needed, df_live], axis=1)

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 300)

# -----------------------------
# Print everything
# -----------------------------
print(title)
print(df)

# -----------------------------
# Shutdown MT5
# -----------------------------
mt5.shutdown()


How much 'distance needed' equivalent of 1 unit risk (account's money as unit. It's 1 USC (cent, for cent accounts) or 1 USD (dollar, for dollar accounts) in Exness Broker) if using a 0.01 volume lot size.
     symbol  needed_tick_points  needed_pips_points  needed_symbol_price_points         bid         ask  spread_tick_points  spread_price_points  spread_pips_points  digits  volume_min  volume_max  volume_step   swap_long  swap_short
0   BTCUSDc               10000          1000.00000                   100.00000 86838.32000 86856.32000                1800             18.00000           180.00000       2     0.01000  1000.00000      0.01000 -1551.10000     0.00000
1   AUDCADc                 138            13.80000                     0.00138     0.90997     0.91019                  22              0.00022             2.20000       5     0.01000   200.00000      0.01000    -1.40000    -7.40000
2   AUDCHFc                  80             8.00000                     0.00080     0.52573 

True

In [71]:
import MetaTrader5 as mt5
import pandas as pd

# -----------------------------
# Title
# -----------------------------
title = "For  needed_tick_points, needed_pips_points, and needed_symbol_price_points data is about: {How much 'distance needed' equivalent of 1 unit risk (account's money as unit. It's 1 USC (cent, for cent accounts) or 1 USD (dollar, for dollar accounts) in Exness Broker) if using a 0.01 volume lot size.}"

# -----------------------------
# Initialize MT5
# -----------------------------
if not mt5.initialize():
    raise RuntimeError(f"MT5 init failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None or "Exness" not in account.company:
    mt5.shutdown()
    raise RuntimeError("MT5 is not connected to Exness")

# -----------------------------
# Pre-calculated needed distances
# -----------------------------
symbols_needed = {
    "symbol": [
        "BTCUSDc","AUDCADc","AUDCHFc","AUDJPYc","AUDNZDc","AUDUSDc","CADJPYc","CHFJPYc",
        "EURAUDc","EURCADc","EURCHFc","EURGBPc","EURJPYc","EURNZDc","EURUSDc","GBPAUDc",
        "GBPCADc","GBPCHFc","GBPJPYc","GBPNZDc","GBPUSDc","NZDJPYc","NZDUSDc","USDCADc",
        "USDCHFc","USDHKDc","USDJPYc","XAGUSDc","XAUUSDc"
    ],
    "needed_tick_points": [
        10000,138,80,156,173,100,156,156,151,138,80,75,156,174,100,151,138,80,156,174,100,156,100,138,80,778,156,20,1000
    ],
    "needed_pips_points": [
        1000,13.8,8,15.6,17.3,10,15.6,15.6,15.1,13.8,8,7.5,15.6,17.4,10,15.1,13.8,8,15.6,17.4,10,15.6,10,13.8,8,77.8,15.6,2,100
    ],
    "needed_symbol_price_points": [
        100,0.00138,0.0008,0.156,0.00173,0.001,0.156,0.156,0.00151,0.00138,0.0008,0.00075,0.156,0.00174,0.001,0.00151,
        0.00138,0.0008,0.156,0.00174,0.001,0.156,0.001,0.00138,0.0008,0.00778,0.156,0.02,1
    ]
}

df_needed = pd.DataFrame(symbols_needed)

# -----------------------------
# Add live MT5 data including requested columns
# -----------------------------
live_data = []

for i, sym in enumerate(df_needed["symbol"]):
    info = mt5.symbol_info(sym)
    tick = mt5.symbol_info_tick(sym)

    if info is None or tick is None:
        live_data.append({
            "bid": None,
            "ask": None,
            "spread_tick_points": None,
            "spread_price_points": None,
            "spread_pips_points": None,
            "digits": None,
            "volume_min": None,
            "volume_max": None,
            "volume_step": None,
            "swap_long": None,
            "swap_short": None
        })
        continue

    digits = info.digits
    bid = tick.bid
    ask = tick.ask
    spread_tick = info.spread if info.spread is not None else (ask - bid if ask and bid else None)
    spread_price = (ask - bid) if ask and bid else None

    # Calculate spread in pips points
    pip_value = df_needed.loc[i, "needed_symbol_price_points"] / df_needed.loc[i, "needed_pips_points"]
    spread_pips_points = spread_price / pip_value if spread_price is not None else None

    # Format all price-related values based on digits
    bid_fmt = f"{bid:.{digits}f}" if bid is not None else None
    ask_fmt = f"{ask:.{digits}f}" if ask is not None else None
    spread_price_fmt = f"{spread_price:.{digits}f}" if spread_price is not None else None
    needed_symbol_price_fmt = f"{df_needed.loc[i, 'needed_symbol_price_points']:.{digits}f}"

    live_data.append({
        "bid": bid_fmt,
        "ask": ask_fmt,
        "spread_tick_points": spread_tick,
        "spread_pips_points": spread_pips_points,
        "spread_price_points": spread_price_fmt,
        "digits": digits,
        "volume_min": info.volume_min,
        "volume_max": info.volume_max,
        "volume_step": info.volume_step,
        "swap_long": info.swap_long,
        "swap_short": info.swap_short,
        "needed_symbol_price_points": needed_symbol_price_fmt
    })

df_live = pd.DataFrame(live_data)

# -----------------------------
# Combine pre-calculated distances with live data
# -----------------------------
df = pd.concat([df_needed.drop(columns="needed_symbol_price_points"), df_live], axis=1)

# -----------------------------
# Rearrange columns
# -----------------------------
column_order = [
    "symbol",
    "needed_tick_points",
    "needed_pips_points",
    "needed_symbol_price_points",
    "bid",
    "ask",
    "spread_tick_points",
    "spread_pips_points",
    "spread_price_points",
    "digits",
    "volume_min",
    "volume_max",
    "volume_step",
    "swap_long",
    "swap_short"
]

df = df[column_order]

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 300)

# -----------------------------
# Print everything
# -----------------------------
print(title)
print(df)

# -----------------------------
# Shutdown MT5
# -----------------------------
mt5.shutdown()


For  needed_tick_points, needed_pips_points, and needed_symbol_price_points data is about: {How much 'distance needed' equivalent of 1 unit risk (account's money as unit. It's 1 USC (cent, for cent accounts) or 1 USD (dollar, for dollar accounts) in Exness Broker) if using a 0.01 volume lot size.}
     symbol  needed_tick_points  needed_pips_points needed_symbol_price_points       bid       ask  spread_tick_points  spread_pips_points spread_price_points  digits  volume_min  volume_max  volume_step   swap_long  swap_short
0   BTCUSDc               10000          1000.00000                     100.00  86962.57  86980.57                1800           180.00000               18.00       2     0.01000  1000.00000      0.01000 -1551.10000     0.00000
1   AUDCADc                 138            13.80000                    0.00138   0.91000   0.91022                  22             2.20000             0.00022       5     0.01000   200.00000      0.01000    -1.40000    -7.40000
2   AUDCHFc      

True

In [73]:
import MetaTrader5 as mt5
import pandas as pd

# -----------------------------
# Title
# -----------------------------
title = "For  needed_tick_points, needed_pips_points, and needed_symbol_price_points data is about: {How much 'distance needed' equivalent of 1 unit risk (account's money as unit. It's 1 USC (cent, for cent accounts) or 1 USD (dollar, for dollar accounts) in Exness Broker) if using a 0.01 volume lot size.}"

# -----------------------------
# Initialize MT5
# -----------------------------
if not mt5.initialize():
    raise RuntimeError(f"MT5 init failed: {mt5.last_error()}")

account = mt5.account_info()
if account is None or "Exness" not in account.company:
    mt5.shutdown()
    raise RuntimeError("MT5 is not connected to Exness")

# -----------------------------
# Pre-calculated needed distances
# -----------------------------
symbols_needed = {
    "symbol": [
        "BTCUSDc","AUDCADc","AUDCHFc","AUDJPYc","AUDNZDc","AUDUSDc","CADJPYc","CHFJPYc",
        "EURAUDc","EURCADc","EURCHFc","EURGBPc","EURJPYc","EURNZDc","EURUSDc","GBPAUDc",
        "GBPCADc","GBPCHFc","GBPJPYc","GBPNZDc","GBPUSDc","NZDJPYc","NZDUSDc","USDCADc",
        "USDCHFc","USDHKDc","USDJPYc","XAGUSDc","XAUUSDc"
    ],
    "needed_tick_points": [
        10000,138,80,156,173,100,156,156,151,138,80,75,156,174,100,151,138,80,156,174,100,156,100,138,80,778,156,20,1000
    ],
    "needed_pips_points": [
        1000,13.8,8,15.6,17.3,10,15.6,15.6,15.1,13.8,8,7.5,15.6,17.4,10,15.1,13.8,8,15.6,17.4,10,15.6,10,13.8,8,77.8,15.6,2,100
    ],
    "needed_symbol_price_points": [
        100,0.00138,0.0008,0.156,0.00173,0.001,0.156,0.156,0.00151,0.00138,0.0008,0.00075,0.156,0.00174,0.001,0.00151,
        0.00138,0.0008,0.156,0.00174,0.001,0.156,0.001,0.00138,0.0008,0.00778,0.156,0.02,1
    ]
}

df_needed = pd.DataFrame(symbols_needed)

# -----------------------------
# Add live MT5 data including requested columns
# -----------------------------
live_data = []

for i, sym in enumerate(df_needed["symbol"]):
    info = mt5.symbol_info(sym)
    tick = mt5.symbol_info_tick(sym)

    if info is None or tick is None:
        live_data.append({
            "bid": None,
            "ask": None,
            "spread_tick_points": None,
            "spread_price_points": None,
            "spread_pips_points": None,
            "digits": None,
            "volume_min": None,
            "volume_max": None,
            "volume_step": None,
            "swap_long": None,
            "swap_short": None,
            "needed_symbol_price_points": None,
            "price_difference": None
        })
        continue

    digits = info.digits
    bid = tick.bid
    ask = tick.ask
    spread_tick = info.spread if info.spread is not None else (ask - bid if ask and bid else None)
    spread_price = (ask - bid) if ask and bid else None

    # Calculate spread in pips points
    pip_value = df_needed.loc[i, "needed_symbol_price_points"] / df_needed.loc[i, "needed_pips_points"]
    spread_pips_points = spread_price / pip_value if spread_price is not None else None

    # Format all price-related values based on digits
    bid_fmt = f"{bid:.{digits}f}" if bid is not None else None
    ask_fmt = f"{ask:.{digits}f}" if ask is not None else None
    spread_price_fmt = f"{spread_price:.{digits}f}" if spread_price is not None else None
    needed_symbol_price_fmt = f"{df_needed.loc[i, 'needed_symbol_price_points']:.{digits}f}"

    # Calculate price_difference
    price_difference = spread_price / df_needed.loc[i, "needed_symbol_price_points"] if spread_price is not None else None

    live_data.append({
        "bid": bid_fmt,
        "ask": ask_fmt,
        "spread_tick_points": spread_tick,
        "spread_pips_points": spread_pips_points,
        "spread_price_points": spread_price_fmt,
        "digits": digits,
        "volume_min": info.volume_min,
        "volume_max": info.volume_max,
        "volume_step": info.volume_step,
        "swap_long": info.swap_long,
        "swap_short": info.swap_short,
        "needed_symbol_price_points": needed_symbol_price_fmt,
        "price_difference": price_difference
    })

df_live = pd.DataFrame(live_data)

# -----------------------------
# Combine pre-calculated distances with live data
# -----------------------------
df = pd.concat([df_needed.drop(columns="needed_symbol_price_points"), df_live], axis=1)

# -----------------------------
# Rearrange columns
# -----------------------------
column_order = [
    "symbol",
    "needed_tick_points",
    "needed_pips_points",
    "needed_symbol_price_points",
    "bid",
    "ask",
    "spread_tick_points",
    "spread_pips_points",
    "spread_price_points",
    "price_difference",
    "digits",
    "volume_min",
    "volume_max",
    "volume_step",
    "swap_long",
    "swap_short"
]

df = df[column_order]

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 300)

# -----------------------------
# Print everything
# -----------------------------
print(title)
print(df)

# -----------------------------
# Shutdown MT5
# -----------------------------
mt5.shutdown()


For  needed_tick_points, needed_pips_points, and needed_symbol_price_points data is about: {How much 'distance needed' equivalent of 1 unit risk (account's money as unit. It's 1 USC (cent, for cent accounts) or 1 USD (dollar, for dollar accounts) in Exness Broker) if using a 0.01 volume lot size.}
     symbol  needed_tick_points  needed_pips_points needed_symbol_price_points       bid       ask  spread_tick_points  spread_pips_points spread_price_points  price_difference  digits  volume_min  volume_max  volume_step   swap_long  swap_short
0   BTCUSDc               10000          1000.00000                     100.00  87128.41  87146.41                1800           180.00000               18.00           0.18000       2     0.01000  1000.00000      0.01000 -1551.10000     0.00000
1   AUDCADc                 138            13.80000                    0.00138   0.91007   0.91029                  22             2.20000             0.00022           0.15942       5     0.01000   200.00000 

True